In [ ]:
import numpy as np
from nltk.tokenize import word_tokenize
from collections import defaultdict 
import pandas as pd
import nltk
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt 
from sklearn.metrics import accuracy_score

In [ ]:
df=pd.read_csv("temp_data3.csv")
df=df.iloc[:4000,:]

In [ ]:
data=df["cleaned_text"].apply(lambda x: " ".join(x.lower() for x in x.split()))
data=data.str.replace("[^\w\s]","")#harf dışı simgeler,noktalama
data= data.str.replace(" \d+", " ")#sayılar 
from nltk.corpus import stopwords
nltk.download("stopwords")
duraklama_kelimeleri = stopwords.words()
data=pd.DataFrame(data,columns=["cleaned_text"])
data=data["cleaned_text"].apply(lambda x: " ".join(x for x in x.split() if x not in duraklama_kelimeleri))

In [ ]:
data=pd.DataFrame(data,columns=["cleaned_text"])
cleaned_text = data['cleaned_text'].values
cleaned_text=np.array(cleaned_text)

In [ ]:
name=["good","bad","negative"]
data_label=[]
sayac=0
for sentences in cleaned_text:
    for word in sentences.split():
        if word not in name:
            sayac=0
        elif(word==name[0]):
            sayac=1
            break
        elif(word==name[1]):
            sayac=2
            break
        elif(word==name[2]):
            sayac=3
            break
    if(sayac==0):
        data_label.append(0)
    elif(sayac==1):
        data_label.append(1)
    elif(sayac==2):
        data_label.append(2)
    elif(sayac==3):
        data_label.append(3)
            

In [ ]:
data_label=pd.DataFrame(data_label,columns=["Labels"])
dataf=pd.concat([data,data_label],axis=1)
dataf

In [ ]:
nltk.download('punkt')
#clean the corpus.
sentences = []
vocab = []
for sent in cleaned_text:
    x = word_tokenize(sent)
    sentence = [w.lower() for w in x if w.isalpha() ]
    sentences.append(sentence)
    for word in sentence:
        if word not in vocab:
            vocab.append(word)
len_vector = len(vocab)

In [ ]:
index_word = {}
i = 0
for word in vocab:
    index_word[word] = i 
    i += 1

In [ ]:
count_dict = defaultdict(int)
vec = np.zeros(len_vector)
for item in sentences[1]:
    count_dict[item] += 1
for key,item in count_dict.items():
    vec[index_word[key]] = item

In [ ]:
def bag_of_words(sent):
    count_dict = defaultdict(int)
    vec = np.zeros(len_vector)
    for item in sent:
        count_dict[item] += 1
    for key,item in count_dict.items():
        vec[index_word[key]] = item
    return vec   

In [ ]:
tablo = np.zeros((len(sentences),len(vocab)))
tablo =pd.DataFrame(tablo,columns=vocab)
i=0
j=0
for i in range(len(vocab)):
    if i<len(sentences):
        vector = bag_of_words(sentences[j])
        tablo.loc[i] = vector
        j+=1
vector = vector.astype("float64")
tablo = tablo.astype("float64")

In [ ]:
labels=dataf["Labels"].values
train_sentences, test_sentences, train_labels, test_labels = train_test_split(tablo, labels, stratify = labels)

In [ ]:
weights = np.random.rand(4, train_sentences.shape[1])
biases = np.random.rand(4, 1)

In [ ]:
def linearPredict(featureMat, weights, biases):
 
    logitScores = np.array([np.empty([4]) for i in range(featureMat.shape[0])]) # her yorumun 4 etiket için olasılığı
    
    for i in range(featureMat.shape[0]): 
        logitScores[i] = (weights.dot(featureMat[i].reshape(-1,1)) + biases).reshape(-1) 
    
    return logitScores

In [ ]:
features = train_sentences.to_numpy()
logitTest = linearPredict(features, weights,biases )
logitTest.shape

In [ ]:
def softmaxNormalizer(logitMatrix):
  
    probabilities = np.array([np.empty([4]) for i in range(logitMatrix.shape[0])]) #(veri uzunluğu kadar,4)
    for i in range(logitMatrix.shape[0]):
        exp = np.exp(logitMatrix[i]) 
        sumOfArr = np.sum(exp) #4 adet üslü yazılan olasılıkların değerlerini toplar
        probabilities[i] = exp/sumOfArr #4 adet üslü yazılan olasılıklar toplamlarına bölünür
    return probabilities

In [ ]:
def multinomialLogReg(features, weights, biases):
    
    logitScores = linearPredict(features, weights, biases)#her yorumun 4 etiket için olasılıkları tutulur
    probabilities = softmaxNormalizer(logitScores)
    predictions = np.array([np.argmax(i) for i in probabilities]) #olasılığı en yüksek olan ile etiketlendirilir
    return probabilities, predictions

In [ ]:
probabilities, predictions = multinomialLogReg(features, weights, biases)
print(probabilities.shape)
print(predictions)

In [ ]:
def crossEntropyLoss(probabilities, target):
    
    n_samples = probabilities.shape[0]
    CELoss = 0
    for sample, i in zip(probabilities, target):
        CELoss += -np.log(sample[i])#gerçek etiketlerin olasılıkları toplanır
    CELoss /= n_samples#ort değer için veri sayısına bölünür
    return CELoss   

In [ ]:
def stochGradDes(learning_rate, epochs, target, features, weights, biases):

    weights = np.random.rand(4, train_sentences.shape[1])
    biases = np.random.rand(4, 1)
    target = target.astype(int)
    loss_list = np.array([]) 
    loss_list2 = np.array([])
    acc_test=np.array([])
    acc_train=np.array([])
    iterations=np.array([])
    size=features.shape[0]
    for i in range(epochs):
        for j in range((size-1)//100+1):
            start_i = j*100
            end_i = start_i + 100
            xb = features[start_i:end_i]
            yb = target[start_i:end_i]
            probabilities, trainPredictions = multinomialLogReg(xb, weights, biases) #yorumların 4 etiket için olasılıkları hesaplanır
            probabilities[np.arange(xb.shape[0]),yb] -= 1 # 3000 yorumun doğru etiketinin olasılığı-1
            grad_weight = probabilities.T.dot(xb) 
            grad_biases = np.sum(probabilities, axis = 0).reshape(-1,1) 
            weights -= (learning_rate * grad_weight)
            biases -= (learning_rate * grad_biases)
            
        probabilities, trainPredictions = multinomialLogReg(features, weights, biases) 
        acc_train=np.append(acc_train,accuracy_score(target,trainPredictions))
        Loss = crossEntropyLoss(probabilities, target) 
        loss_list = np.append(loss_list, Loss) 
        
        
        testProbabilities, testPredictions = multinomialLogReg(test_sentences.to_numpy(), weights, biases)
        acc_test=np.append(acc_test,accuracy_score(test_labels,testPredictions))
        Loss2 = crossEntropyLoss(testProbabilities, test_labels) 
        loss_list2 = np.append(loss_list2, Loss2) 
        iterations=np.append(iterations,i)
        
    return weights, biases, loss_list,loss_list2,acc_test,acc_train,iterations

In [ ]:
updatedWeights, updatedBiases, loss_list,loss_list2,acc_test,acc_train,iterations = stochGradDes(0.02, 100, train_labels, train_sentences.to_numpy(), weights, biases)

In [ ]:
plt.scatter(iterations,acc_train)  
plt.xlabel( 'iterations' ) 
plt.ylabel( 'accuracy_tr' )  
plt.title("0.02-100-Bow-AccTrain_Multi")
plt.show()

In [ ]:
plt.scatter(iterations,acc_test)  
plt.xlabel( 'iterations' ) 
plt.ylabel( 'accuracy_te' )  
plt.title("0.02-100-TfIdf-AccTest_Multi")
plt.show()

In [ ]:
plt.scatter(iterations,loss_list)  
plt.xlabel( 'iterations' ) 
plt.ylabel( 'loss_train' )  
plt.title("0.02-100-TfIdf-LossTrain_Multi")
plt.show()

In [ ]:
plt.scatter(iterations,loss_list2)  
plt.xlabel( 'iterations' ) 
plt.ylabel( 'loss_test' )  
plt.title("0.02-100-TfIdf-LossTest_Multi")
plt.show()